In [1]:
import os
import numpy as np
import unyt
import swiftsimio as sw
from swiftgalaxy import SWIFTGalaxy, SOAP
from swiftsimio.visualisation.projection import project_gas, project_pixel_grid
from swiftsimio.visualisation import generate_smoothing_lengths
import matplotlib.pyplot as plt
import matplotlib.colors

In [17]:
colibre_base_path = "cosma_files/"
simulation_dir = "L012_m6/THERMAL_AGN_m6"
soap_catalogue_file = os.path.join(
    colibre_base_path,
    simulation_dir,
    "SOAP/halo_properties_0127.hdf5",
)


virtual_snapshot_file = os.path.join(
    colibre_base_path, simulation_dir, "SOAP/colibre_with_SOAP_membership_0127.hdf5"
)

In [19]:
sd = sw.SWIFTDataset(soap_catalogue_file)
#m200c = sd.spherical_overdensity_200_crit.total_mass
#candidates = np.argwhere(
#    np.logical_and(m200c > 1e11 * u.Msun, m200c < 2e11 * u.Msun)
#).squeeze()
# unnecessary, since I determine my chosen index elsewhere

chosen_halo_index = 11956

In [20]:
sg = SWIFTGalaxy(
    virtual_snapshot_file,
    SOAP(
        soap_catalogue_file,
        soap_index=chosen_halo_index,
    ),
)

In [21]:
dir(sg)

['__annotations__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_append_to_coordinate_like_transform',
 '_append_to_velocity_like_transform',
 '_coordinate_like_transform',
 '_copyinit',
 '_data_copy',
 '_extra_mask',
 '_initialised',
 '_particle_dataset_helpers',
 '_spatial_mask',
 '_transform',
 '_translate',
 '_velocity_like_transform',
 '_void_derived_coordinates',
 '_warn_on_read',
 'black_holes',
 'boost',
 'centre',
 'coordinates_dataset_name',
 'create_datasets',
 'dark_matter',
 'filename',
 'gas',
 'get_metadata',
 'get_units',
 'halo_catalogue',
 'id_particle_dataset_name',
 'mask',
 'mask_p

In [33]:
sg.gas.coordinates

cosmo_array([], shape=(0, 3), dtype=float64, units='Mpc', comoving='True', cosmo_factor='a**1.0 at a=0.9999999999999997', valid_transform='True')

In [24]:
file = sw.load("cosma_files/L012_m6/THERMAL_AGN_m6/SOAP/colibre_with_SOAP_membership_0127.hdf5")

In [27]:
dir(file)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'black_holes',
 'create_datasets',
 'dark_matter',
 'filename',
 'gas',
 'get_metadata',
 'get_units',
 'mask',
 'metadata',
 'stars',
 'units']

In [28]:
dir(file.metadata)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'a',
 'black_holes_properties',
 'boxsize',
 'code',
 'code_info',
 'compiler_info',
 'cosmology',
 'cosmology_raw',
 'dark_matter_properties',
 'diffusion_info',
 'dimension',
 'extract_cosmology',
 'filename',
 'gas_gamma',
 'gas_properties',
 'get_mapping_metadata',
 'get_metadata',
 'get_named_column_metadata',
 'get_nice_name',
 'grain_to_element_mapping',
 'gravity_scheme',
 'handle',
 'has_type',
 'header',
 'homogeneous_arrays',
 'hydro_info',
 'hydro_scheme',
 'initial_mass_table',
 'internal_code_units',
 'library_info',
 'l